<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/HS/DART_API_%EC%83%81%EC%9E%A5%EA%B8%B0%EC%97%85_%EC%A0%84%EC%B2%B4_%EC%BB%AC%EB%9F%BC_%EC%A1%B0%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 마운트
import google.colab.drive
google.colab.drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/실전프로젝트

/content/drive/MyDrive/Colab Notebooks/실전프로젝트


In [1]:
!pip install -q --upgrade \
  torch==2.0.1+cpu torchvision==0.15.2+cpu torchaudio==2.0.2+cpu \
  -f https://download.pytorch.org/whl/torch_stable.html

!pip install -q sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 40.1 MB/s eta 0:00:00


In [5]:
# Colab 셀에 붙여넣고 실행
!pip install -q --upgrade transformers>=4.33.1 sentence-transformers>=2.2.2


In [4]:
!pip install -q --upgrade numpy==1.24.4
!pip install -q --upgrade \
  --index-url https://download.pytorch.org/whl/cpu \
  torch==2.1.0+cpu torchvision==0.16.0+cpu torchaudio==2.1.1+cpu
!pip install -q --upgrade sentence-transformers transformers>=4.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 33.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incomp

In [4]:
# Colab 혹은 pip 환경에서
!pip install -q torch torchvision torchaudio sentence-transformers


In [36]:
import requests, zipfile, io
import glob, os
import xml.etree.ElementTree as ET
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ──────────────────────────────────────────────
# 설정
API_KEY = "845166f5401aeebbed295c86b5f47621b51f520a"  # 본인 키
YEAR    = "2023"
REPORT  = "11011"  # 연간보고서
FS_DIV  = "OFS"    # 개별재무제표

# ──────────────────────────────────────────────
def get_all_corp_codes(api_key: str) -> list[str]:
    """DART corpCode.xml ZIP을 받아 풀고, 모든 <list><corp_code> 태그를 읽어 반환."""
    url = "https://opendart.fss.or.kr/api/corpCode.xml"
    res = requests.get(url, params={"crtfc_key": api_key})
    with zipfile.ZipFile(io.BytesIO(res.content)) as zf:
        zf.extractall()  # 현재 폴더에 CORPCODE.xml이 풀립니다
    tree = ET.parse("CORPCODE.xml")
    root = tree.getroot()
    codes = [e.findtext("corp_code").strip() for e in root.findall(".//list") if e.findtext("corp_code")]
    return list(set(codes))  # 중복 제거

def fetch_account_names(codes_batch: list[str]) -> list[str]:
    """fnlttMultiAcnt.json으로 OFS 계정명만 뽑아서 반환."""
    url = "https://opendart.fss.or.kr/api/fnlttMultiAcnt.json"
    params = {
        "crtfc_key": API_KEY,
        "corp_code": ",".join(codes_batch),
        "bsns_year": YEAR,
        "reprt_code": REPORT
    }
    data = requests.get(url, params=params).json()
    if data.get("status") != "000":
        return []
    df = pd.DataFrame(data.get("list", []))
    return df[df["fs_div"] == FS_DIV]["account_nm"].unique().tolist()

def chunked(lst: list, n: int):
    """리스트를 n개씩 잘라주는 제너레이터."""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

# ──────────────────────────────────────────────
def main():
    # 1) 전체 corp_code 뽑기
    corp_codes = get_all_corp_codes(API_KEY)
    print(f"전체 기업 코드: {len(corp_codes)}개")

    # 2) 배치별 계정명 수집
    all_accounts = set()
    for batch in chunked(corp_codes, 100):
        names = fetch_account_names(batch)
        print(f"  • 배치 {len(batch)}개 → {len(names)}개 계정명")
        all_accounts.update(names)
    account_names = list(all_accounts)
    print(f"총 {len(account_names)}개 고유 계정명 확보")

    # 3) SBERT 임베딩
    model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
    embs  = model.encode(account_names, convert_to_numpy=True)

    # 4) 코사인 유사도 계산 & Top 50 페어
    sim = cosine_similarity(embs)
    pairs = [
        (account_names[i], account_names[j], sim[i,j])
        for i, j in itertools.combinations(range(len(account_names)), 2)
    ]
    top50 = sorted(pairs, key=lambda x: x[2], reverse=True)[:50]
    df_top50 = pd.DataFrame(top50, columns=["Account1","Account2","Similarity"])
    print(df_top50.style.format({"Similarity":"{:.3f}"}))

if __name__ == "__main__":
    main()


전체 기업 코드: 112899개
  • 배치 100개 → 16개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 0개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 0개 계정명
  • 배치 100개 → 0개 계정명
  • 배치 100개 → 16개 계정명
  • 배치 100개 → 18개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 22개 계정명
  • 배치 100개 → 16개 계정명
  • 배치 100개 → 0개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 25개 계정명
  • 배치 100개 → 0개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 16개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 16개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 0개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 16개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 15개 계정명
  • 배치 100개 → 18개 계정

KeyboardInterrupt: 

In [3]:
import requests, zipfile, io
import xml.etree.ElementTree as ET
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ──────────────────────────────────────────────
# 설정
API_KEY = "845166f5401aeebbed295c86b5f47621b51f520a"  # DART API 키
YEAR    = "2023"
REPORT  = "11011"  # 연간보고서
FS_DIV  = "OFS"    # 개별재무제표
SUBSET_SIZE = 100    # 처리할 기업 개수 제한
BATCH_SIZE  = 50     # 한 번에 호출할 기업 수

# ──────────────────────────────────────────────

def get_all_corp_codes(api_key: str) -> list[str]:
    """전체 기업코드 리스트를 반환합니다."""
    url = "https://opendart.fss.or.kr/api/corpCode.xml"
    res = requests.get(url, params={"crtfc_key": api_key})
    with zipfile.ZipFile(io.BytesIO(res.content)) as zf:
        zf.extractall()
    tree = ET.parse("CORPCODE.xml")
    codes = [e.findtext("corp_code").strip() for e in tree.getroot().findall('.//list')]
    return list(dict.fromkeys(codes))  # 중복 제거 유지 순서


def fetch_account_names(codes_batch: list[str]) -> list[str]:
    """주어진 기업코드 배치에서 OFS 계정명만 추출"""
    url = "https://opendart.fss.or.kr/api/fnlttMultiAcnt.json"
    params = {
        "crtfc_key": API_KEY,
        "corp_code": ",".join(codes_batch),
        "bsns_year": YEAR,
        "reprt_code": REPORT
    }
    data = requests.get(url, params=params).json()
    if data.get("status") != "000":
        return []
    df = pd.DataFrame(data.get("list", []))
    return df[df["fs_div"] == FS_DIV]["account_nm"].unique().tolist()


def chunked(lst: list, size: int):
    """리스트를 size씩 분할"""
    for i in range(0, len(lst), size):
        yield lst[i:i+size]


# ──────────────────────────────────────────────

def main():
    # 1) 전체 기업코드 로드 및 제한
    all_codes = get_all_corp_codes(API_KEY)
    corp_codes = all_codes[:SUBSET_SIZE]
    print(f"전체 기업 {len(all_codes)}개 중 처음 {len(corp_codes)}개만 처리")

    # 2) 배치별 계정명 수집
    all_accounts = set()
    for batch in chunked(corp_codes, BATCH_SIZE):
        names = fetch_account_names(batch)
        print(f"배치 {len(batch)}개 → {len(names)}개 계정명 수집")
        all_accounts.update(names)
    account_names = list(all_accounts)
    print(f"총 {len(account_names)}개 고유 계정명 확보")

    # 3) 임베딩 & 유사도 계산
    model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
    embs  = model.encode(account_names, convert_to_numpy=True)
    sim   = cosine_similarity(embs)

    # 4) Top 50 유사도 페어
    pairs = [
        (account_names[i], account_names[j], sim[i,j])
        for i,j in itertools.combinations(range(len(account_names)),2)
    ]
    top50 = sorted(pairs, key=lambda x: x[2], reverse=True)[:50]
    df_top50 = pd.DataFrame(top50, columns=["Account1","Account2","Similarity"])
    print(df_top50.to_string(index=False))


if __name__ == "__main__":
    main()


Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.1+cpu

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in star

NameError: name 'LRScheduler' is not defined